# Clustering - Customer Segmentaiton
## Part 2. Data Wrangling - Customer Level


<a id = 'toc'></a>
**Table of Contents**

1. [create customer level features](#feature)
    - number of orders (InvoiceNo) per customer
    - number of products
    - number of unique products
    - max/min UnitPrice
    - total Sales
    - max/min sales per customer
    - Average Sales per InvoiceNo


2. [save data file](#save)

In [1]:
# load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

In [2]:
dtypes = {'InvoiceNo': str,
          'StockCode': str,
          'Description': str,
          'Quantity': 'int64',
          'UnitPrice': 'float64',
          'CustomerID': str,
          'Country': str,
          'Sales': 'float64'
         }

In [3]:
cleaned_tx_df = pd.read_csv('../data/cleaned_tx_df.csv', dtype = dtypes, parse_dates = ['InvoiceDate'])

In [4]:
cleaned_tx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33698 entries, 0 to 33697
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    33698 non-null  object        
 1   StockCode    33698 non-null  object        
 2   Description  33698 non-null  object        
 3   Quantity     33698 non-null  int64         
 4   InvoiceDate  33698 non-null  datetime64[ns]
 5   UnitPrice    33698 non-null  float64       
 6   CustomerID   33698 non-null  object        
 7   Country      33698 non-null  object        
 8   Sales        33698 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 2.3+ MB


In [5]:
cleaned_tx_df.describe(include = 'all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
count,33698,33698,33698,33698.000000,33698,33698.000000,33698,33698,33698.000000
unique,1536,2574,2639,NaN,1523,NaN,414,36,NaN
top,570672,POST,POSTAGE,NaN,2011-10-11 14:52:00,NaN,12681,Germany,NaN
freq,259,1055,1055,NaN,259,NaN,638,9040,NaN
first,NaN,NaN,NaN,NaN,2010-12-01 08:45:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,14.797110,NaN,4.480351,NaN,NaN,27.796638
std,NaN,NaN,NaN,31.690705,NaN,47.179203,NaN,NaN,69.149712
min,NaN,NaN,NaN,1.000000,NaN,0.040000,NaN,NaN,0.190000
25%,NaN,NaN,NaN,5.000000,NaN,1.250000,NaN,NaN,12.600000


[back to top](#toc)
<a id = 'feature'></a>
## 1. Create customer level features:
- ~~number of orders (InvoiceNo) per customer~~
- ~~number of products~~
- ~~number of unique products~~
- ~~max/min UnitPrice~~
- ~~total Sales~~
- ~~max/min sales per customer~~
- ~~Average Sales per InvoiceNo~~

In [6]:
df = cleaned_tx_df.copy()

In [7]:
# feature - numberOfOrders per customer
orders = df.groupby('CustomerID')['InvoiceNo'].agg(['nunique'])
orders.columns = ['numberOfOrders']
orders.head()

,numberOfOrders
CustomerID,
12347,7
12348,4
12349,1
12350,1
12352,8


In [8]:
# features - numberOfProducts, numberOfUniqueProducts
products = df.groupby('CustomerID')['StockCode'].agg(['count', 'nunique'])
products.columns = ['numberOfProducts', 'numberOfUniqueProducts']
products.head()

,numberOfProducts,numberOfUniqueProducts
CustomerID,,
12347,182,103
12348,31,22
12349,73,73
12350,17,17
12352,85,59


In [9]:
(products.numberOfProducts >= products.numberOfUniqueProducts).all()

True

In [10]:
# feature - highest and lowest unit price items customer has ever purchased
minMaxUnitPrice = df.groupby('CustomerID')['UnitPrice'].agg(['max','min'])
minMaxUnitPrice.columns = ['maxUnitPrice', 'minUnitPrice']
minMaxUnitPrice.head()

,maxUnitPrice,minUnitPrice
CustomerID,,
12347,12.75,0.25
12348,40.00,0.29
12349,300.00,0.42
12350,40.00,0.85
12352,376.50,0.65


In [11]:
# feature - total sales to a specific customer
sales = df.groupby('CustomerID')['Sales'].agg(['sum'])
sales.columns = ['totalSales']
sales.head()

,totalSales
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
12352,2506.04


In [12]:
# feature - customer per order behavior
minMaxOrders = df.groupby(['CustomerID', 'InvoiceNo'])['Sales'].agg(['sum'])
minMaxOrders.columns = ['orderSales']
minMaxOrders.head()

orderSales
CustomerID InvoiceNo            
12347      537626         711.79
           542237         475.39
           549222         636.25
           556201         382.52
           562032         584.91

In [13]:
#features - 
minMaxOrders = minMaxOrders.reset_index().groupby('CustomerID')['orderSales'].agg(['min', 'max', 'mean'])
minMaxOrders.columns = ['minOrderSales', 'maxOrderSales', 'meanOrderSales']
minMaxOrders.head()

,minOrderSales,maxOrderSales,meanOrderSales
CustomerID,,,
12347,224.82,1294.32,615.714286
12348,227.44,892.80,449.310000
12349,1757.55,1757.55,1757.550000
12350,334.40,334.40,334.400000
12352,120.33,840.30,313.255000


In [14]:
customer_df = orders.join([products, minMaxUnitPrice, sales, minMaxOrders])
customer_df.shape

(414, 9)

In [15]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 12347 to 17844
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   numberOfOrders          414 non-null    int64  
 1   numberOfProducts        414 non-null    int64  
 2   numberOfUniqueProducts  414 non-null    int64  
 3   maxUnitPrice            414 non-null    float64
 4   minUnitPrice            414 non-null    float64
 5   totalSales              414 non-null    float64
 6   minOrderSales           414 non-null    float64
 7   maxOrderSales           414 non-null    float64
 8   meanOrderSales          414 non-null    float64
dtypes: float64(6), int64(3)
memory usage: 52.3+ KB


In [16]:
customer_df.head()

,numberOfOrders,numberOfProducts,numberOfUniqueProducts,maxUnitPrice,minUnitPrice,totalSales,minOrderSales,maxOrderSales,meanOrderSales
CustomerID,,,,,,,,,
12347,7,182,103,12.75,0.25,4310.00,224.82,1294.32,615.714286
12348,4,31,22,40.00,0.29,1797.24,227.44,892.80,449.310000
12349,1,73,73,300.00,0.42,1757.55,1757.55,1757.55,1757.550000
12350,1,17,17,40.00,0.85,334.40,334.40,334.40,334.400000
12352,8,85,59,376.50,0.65,2506.04,120.33,840.30,313.255000


In [16]:
customer_df.describe()

,numberOfOrders,numberOfProducts,numberOfUniqueProducts,maxUnitPrice,minUnitPrice,totalSales,minOrderSales,maxOrderSales,meanOrderSales
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,3.710145,81.396135,60.625604,60.499855,0.580483,2262.538913,423.700459,898.712874,637.347366
std,4.255524,96.304061,60.383299,299.793532,1.060587,3479.689136,648.729257,1106.085217,740.293974
min,1.000000,1.000000,1.000000,0.850000,0.040000,51.560000,3.000000,51.560000,51.560000
25%,1.000000,21.250000,20.000000,16.950000,0.210000,459.650000,120.510000,325.662500,259.829750
50%,2.000000,45.000000,38.000000,18.000000,0.420000,1025.700000,252.150000,551.615000,417.804000
75%,4.000000,101.000000,82.000000,28.000000,0.550000,2659.962500,460.767500,1053.660000,710.275000
max,32.000000,638.000000,323.000000,4161.060000,15.000000,31906.820000,6207.670000,9341.260000,6207.670000


[back to top](#toc)
<a id = 'save'></a>
## 2. Save Datafile

In [17]:
customer_df.to_csv('../data/analytical_base_table.csv')

In [18]:
df = pd.read_csv('../data/analytical_base_table.csv', float_precision = 'round_trip', index_col = 'CustomerID')

In [19]:
customer_df.index = customer_df.index.astype(int)

In [20]:
(customer_df == df).all().all()

True